<a href="https://colab.research.google.com/github/ShankyMalewar/Asklyne_AI_Backend/blob/main/OR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab cell 1
!pip install pandas networkx tqdm python-dateutil

# If you have Gurobi and license available in Colab uncomment:
# !pip install gurobipy

import os, glob, math
import pandas as pd
import networkx as nx
from datetime import datetime, timedelta
from dateutil import parser
from tqdm import tqdm


In [ ]:
mkdir /content/instance1

In [ ]:
# Robust loader for listOfBases + day_x files
import os, glob
import pandas as pd
from dateutil import parser
from tqdm import tqdm

INSTANCE_DIR = "/content/instance1"  # update if needed

# --- 1) Load bases robustly ---
bases_path = os.path.join(INSTANCE_DIR, "listOfBases.csv")
bases_df = pd.read_csv(bases_path)
# normalize column names
bases_df.columns = bases_df.columns.str.strip().str.lower()
# common name variants mapping
colmap_bases = {}
if 'airport' in bases_df.columns:
    colmap_bases['airport'] = 'airport'
elif 'airports' in bases_df.columns:
    colmap_bases['airport'] = 'airports'
# status variants
for c in ['status','stat','is_base']:
    if c in bases_df.columns:
        colmap_bases['status'] = c
        break
# nbEmployees variants
for c in ['nbemployees','nbEmployees','nb_employees','nbemployees']:
    if c in bases_df.columns:
        colmap_bases['nbemployees'] = c
        break

# If mapping incomplete, try fuzzy matching:
if 'airport' not in colmap_bases:
    for c in bases_df.columns:
        if 'air' in c and 'port' in c:
            colmap_bases['airport'] = c; break
if 'status' not in colmap_bases:
    for c in bases_df.columns:
        if 'stat' in c:
            colmap_bases['status'] = c; break
if 'nbemployees' not in colmap_bases:
    for c in bases_df.columns:
        if 'nb' in c or 'employee' in c:
            colmap_bases['nbemployees'] = c; break

# rename for internal use
bases_df = bases_df.rename(columns={colmap_bases['airport']:'airport',
                                    colmap_bases['status']:'status',
                                    colmap_bases['nbemployees']:'nbemployees'})
# strip whitespace
bases_df['airport'] = bases_df['airport'].astype(str).str.strip()
bases_df['status'] = bases_df['status'].astype(str).str.strip()
# convert status to int safely
bases_df['status'] = bases_df['status'].replace('', '0')  # default if missing
bases_df['status'] = bases_df['status'].astype(int)
print("Bases head (clean):")
print(bases_df.head())
bases = bases_df[bases_df['status'] == 1]['airport'].tolist()
print("Bases (status==1):", len(bases), "example:", bases[:8])

# --- 2) Load day_*.csv files robustly and build flights_df ---
files = sorted(glob.glob(os.path.join(INSTANCE_DIR, "day_*.csv")))
print("Found day files:", files)

flights = []
bad_files = []
expected_keys = ["leg", "dep_air", "dep_date", "dep_time", "arr_air", "arr_date", "arr_time"]
# function to find best matching column by keyword
def find_col(cols, keywords):
    for k in keywords:
        for c in cols:
            if k in c:
                return c
    return None

for fpath in files:
    try:
        df = pd.read_csv(fpath)
        orig_cols = list(df.columns)
        # normalize columns: strip and lower
        df.columns = [c.strip().lower() for c in df.columns.astype(str)]
        cols = df.columns.tolist()

        # try to map columns
        # leg id variants
        leg_col = find_col(cols, ['leg','flight','leg_nb','legno','leg no','leg#'])
        dep_air_col = find_col(cols, ['airport_dep','dep_airport','dep','origin','airport_dep'])
        dep_date_col = find_col(cols, ['date_dep','dep_date','date dep','dep_date'])
        dep_time_col = find_col(cols, ['hour_dep','dep_time','time_dep','dep_time','hour dep'])
        arr_air_col = find_col(cols, ['airport_arr','arr_airport','arr','destination','airport_arr'])
        arr_date_col = find_col(cols, ['date_arr','arr_date','date arr'])
        arr_time_col = find_col(cols, ['hour_arr','arr_time','time_arr','arr_time','hour arr'])

        mapping = {
            'leg': leg_col, 'dep_air': dep_air_col, 'dep_date': dep_date_col,
            'dep_time': dep_time_col, 'arr_air': arr_air_col, 'arr_date': arr_date_col, 'arr_time': arr_time_col
        }
        if any(v is None for v in mapping.values()):
            bad_files.append((fpath, orig_cols))
            continue

        # iterate rows
        for _, r in df.iterrows():
            leg = r[leg_col]
            dep_air = str(r[dep_air_col]).strip()
            arr_air = str(r[arr_air_col]).strip()
            dep_date = str(r[dep_date_col]).strip()
            dep_time = str(r[dep_time_col]).strip()
            arr_date = str(r[arr_date_col]).strip()
            arr_time = str(r[arr_time_col]).strip()
            # some weird blanks -> skip
            if dep_date == '' or dep_time == '' or arr_date == '' or arr_time == '':
                continue
            try:
                dep_dt = parser.parse(dep_date + " " + dep_time)
                arr_dt = parser.parse(arr_date + " " + arr_time)
            except Exception as e:
                # try with replacement for weird formats
                try:
                    dep_dt = parser.parse(dep_date + " " + dep_time.replace('.',':'))
                    arr_dt = parser.parse(arr_date + " " + arr_time.replace('.',':'))
                except Exception:
                    continue
            flights.append({
                "flight_id": str(leg).strip(),
                "dep_airport": dep_air,
                "arr_airport": arr_air,
                "dep_dt": dep_dt,
                "arr_dt": arr_dt,
                "duration_hr": (arr_dt - dep_dt).total_seconds()/3600.0
            })

    except Exception as e:
        bad_files.append((fpath, str(e)))

flights_df = pd.DataFrame(flights)
flights_df = flights_df.sort_values(['dep_dt']).reset_index(drop=True)
print("Total flights loaded:", len(flights_df))
if bad_files:
    print("Warning: some files had unexpected schema or errors. Inspect below:")
    for bf in bad_files:
        print(bf[0], " -> cols/or error:", bf[1])

# quick peek
flights_df.head(10)


Bases head (clean):
  airport  status  nbemployees
0   BASE1       1            7
1    AIR1       0            0
2   BASE2       1           20
3    AIR2       0            0
4    AIR3       0            0
Bases (status==1): 3 example: ['BASE1', 'BASE2', 'BASE3']
Found day files: ['/content/instance1/day_1.csv', '/content/instance1/day_10.csv', '/content/instance1/day_11.csv', '/content/instance1/day_12.csv', '/content/instance1/day_13.csv', '/content/instance1/day_14.csv', '/content/instance1/day_15.csv', '/content/instance1/day_16.csv', '/content/instance1/day_17.csv', '/content/instance1/day_18.csv', '/content/instance1/day_19.csv', '/content/instance1/day_2.csv', '/content/instance1/day_20.csv', '/content/instance1/day_21.csv', '/content/instance1/day_22.csv', '/content/instance1/day_23.csv', '/content/instance1/day_24.csv', '/content/instance1/day_25.csv', '/content/instance1/day_26.csv', '/content/instance1/day_27.csv', '/content/instance1/day_28.csv', '/content/instance1/day_29.

,flight_id,dep_airport,arr_airport,dep_dt,arr_dt,duration_hr
0,LEG_01_0,BASE1,AIR1,2000-01-01 12:00:00,2000-01-01 13:13:00,1.216667
1,LEG_01_30,BASE2,AIR12,2000-01-01 12:59:00,2000-01-01 15:46:00,2.783333
2,LEG_01_1,AIR1,BASE2,2000-01-01 14:05:00,2000-01-01 15:19:00,1.233333
3,LEG_01_18,BASE3,AIR9,2000-01-01 14:05:00,2000-01-01 15:55:00,1.833333
4,LEG_01_6,BASE2,AIR6,2000-01-01 14:21:00,2000-01-01 16:54:00,2.550000
5,LEG_01_11,BASE1,AIR9,2000-01-01 15:24:00,2000-01-01 18:32:00,3.133333
6,LEG_01_31,BASE2,AIR3,2000-01-01 16:13:00,2000-01-01 16:59:00,0.766667
7,LEG_01_28,AIR12,BASE2,2000-01-01 17:00:00,2000-01-01 19:52:00,2.866667
8,LEG_01_15,AIR9,BASE2,2000-01-01 17:05:00,2000-01-01 19:44:00,2.650000
9,LEG_01_8,AIR6,BASE2,2000-01-01 17:40:00,2000-01-01 20:23:00,2.716667


In [ ]:
# === Next step: build adjacency, enumerate pairings, compute cost, save ===
import time, json
from tqdm import tqdm

# Parameters (tune if needed)
MIN_CONNECT_MIN = 45         # minutes
MIN_REST_HOURS = 10          # overnight rest threshold (new duty)
MAX_PAIRING_HOURS = 84       # total hours allowed from first dep to last arr
MAX_PAIRING_FLIGHTS = 12     # max legs per pairing
MAX_PAIRING_DUTIES = 4       # max number of duties (rest cycles)
MAX_ENUM_PAIRINGS = 200000   # safety cap to abort enumeration if too many

# Precompute useful arrays to speed checks
n = len(flights_df)
dep_air = flights_df['dep_airport'].to_list()
arr_air = flights_df['arr_airport'].to_list()
dep_dt  = flights_df['dep_dt'].to_list()
arr_dt  = flights_df['arr_dt'].to_list()

# Build adjacency using time-windowed search (faster than nested loops)
# For each flight i, find candidate j where dep_dt[j] > arr_dt[i] and airports match
from bisect import bisect_left
# Create index of flights sorted by dep_dt for range checks
sorted_by_dep = flights_df.sort_values('dep_dt').reset_index()
dep_sorted_idx = sorted_by_dep['index'].to_list()
dep_times_sorted = sorted_by_dep['dep_dt'].to_list()

def can_connect_idx(i, j):
    # check airports and time gap
    if arr_air[i] != dep_air[j]:
        return False
    gap_min = (dep_dt[j] - arr_dt[i]).total_seconds()/60.0
    if gap_min < MIN_CONNECT_MIN or gap_min < 0:
        return False
    # avoid ridiculously long same-duty gaps
    if gap_min/60.0 > 72:
        return False
    return True

# Build adjacency lists
adj = [[] for _ in range(n)]
# For each i, find earliest candidate j in sorted_by_dep where dep_dt[j] >= arr_dt[i] + MIN_CONNECT
for i in tqdm(range(n), desc="building adjacency"):
    earliest_dt = arr_dt[i] + pd.Timedelta(minutes=MIN_CONNECT_MIN)
    # find first candidate index in dep_times_sorted
    pos = bisect_left(dep_times_sorted, earliest_dt)
    # iterate forward from pos while within a reasonable window (arr_dt + 72h)
    latest_dt = arr_dt[i] + pd.Timedelta(hours=72)
    k = pos
    while k < len(dep_times_sorted) and dep_times_sorted[k] <= latest_dt:
        j = dep_sorted_idx[k]  # original index
        if can_connect_idx(i, j):
            adj[i].append(j)
        k += 1

# Quick adjacency stats
num_edges = sum(len(v) for v in adj)
print("Adjacency built: flights n =", n, ", edges =", num_edges)

# === Enumerate pairings with DFS, bounded by constraints ===
pairings = []
abort = False
start_time = time.time()

def estimate_duties_from_path(path_idx):
    duties = 1
    for a,b in zip(path_idx[:-1], path_idx[1:]):
        gap_h = (dep_dt[b] - arr_dt[a]).total_seconds()/3600.0
        if gap_h >= MIN_REST_HOURS:
            duties += 1
    return duties

def dfs(path):
    global abort, pairings
    if abort:
        return
    first_idx = path[0]; last_idx = path[-1]
    total_hours = (arr_dt[last_idx] - dep_dt[first_idx]).total_seconds()/3600.0
    if total_hours > MAX_PAIRING_HOURS or len(path) > MAX_PAIRING_FLIGHTS:
        return
    duties = estimate_duties_from_path(path)
    if duties <= MAX_PAIRING_DUTIES:
        # pairing must start & end at same base
        start_base = dep_air[first_idx].strip()
        end_base   = arr_air[last_idx].strip()
        if start_base == end_base and start_base in bases:
            pairings.append({
                "flights_idx": path.copy(),
                "start_base": start_base,
                "first_dep": dep_dt[first_idx].isoformat(),
                "last_arr": arr_dt[last_idx].isoformat(),
                "total_hours": total_hours,
                "duties": duties,
                "n_legs": len(path)
            })
            # safety check
            if len(pairings) >= MAX_ENUM_PAIRINGS:
                abort = True
                return
    # expand
    for succ in adj[last_idx]:
        if succ in path:
            continue
        # cheap prune: if succ departure is more than MAX_PAIRING_HOURS after first dep, skip
        if (dep_dt[succ] - dep_dt[first_idx]).total_seconds()/3600.0 > MAX_PAIRING_HOURS:
            continue
        dfs(path + [succ])

# Start DFS from flights that depart from a base
starts = [i for i in range(n) if dep_air[i].strip() in bases]
print("Starting DFS from", len(starts), "base-departure flights")
for s in tqdm(starts, desc="enumerating pairings"):
    dfs([s])
    if abort:
        break

elapsed = time.time() - start_time
print(f"Enumeration finished in {elapsed:.1f}s, pairings found = {len(pairings)}, aborted={abort}")

# Convert to DataFrame, compute cost & credit
pairings_df = pd.DataFrame(pairings)
if pairings_df.shape[0] == 0:
    print("No pairings found — adjust MIN_REST_HOURS / MAX_PAIRING_HOURS or check bases mapping.")
else:
    # compute credit & cost (adjust weights as needed)
    def pairing_cost(row, pay_per_hour=100.0, overnight_cost=200.0):
        return row['total_hours'] * pay_per_hour + row['duties'] * overnight_cost
    pairings_df['credit'] = pairings_df['total_hours']
    pairings_df['cost'] = pairings_df.apply(pairing_cost, axis=1)
    print(pairings_df[['start_base','n_legs','duties','total_hours','cost']].describe())
    display(pairings_df.head(10))

# Save results for later solver steps
# === Save pairings safely (fixed) ===
import pickle, json, os

# save pairings and flights
pairings_pkl = "/content/pairings_instance1.pkl"
flights_pkl  = "/content/flights_instance1.pkl"
pairings_df.to_pickle(pairings_pkl)
flights_df.to_pickle(flights_pkl)
print("Saved pickles:", pairings_pkl, flights_pkl)

# build A: flight_idx (str) -> list(pairing_idx)
A = {}
for p_idx, row in pairings_df.iterrows():
    for f in row['flights_idx']:
        key = str(int(f))    # ensure json-friendly string keys
        A.setdefault(key, []).append(int(p_idx))

# build B: base -> list(pairing_idx)
B = {}
for p_idx, row in pairings_df.iterrows():
    base = str(row['start_base']).strip()
    B.setdefault(base, []).append(int(p_idx))

# save indices as JSON
with open("/content/A_index.json","w") as fh:
    json.dump(A, fh)
with open("/content/B_index.json","w") as fh:
    json.dump(B, fh)

print("Saved A_index.json and B_index.json (flight->pairings and base->pairings)")


building adjacency: 100%|██████████| 1013/1013 [00:00<00:00, 9950.44it/s]


Adjacency built: flights n = 1013 , edges = 14729
Starting DFS from 509 base-departure flights


enumerating pairings:   1%|          | 3/509 [01:37<4:35:14, 32.64s/it]


Enumeration finished in 97.9s, pairings found = 200000, aborted=True
              n_legs         duties    total_hours           cost
count  200000.000000  200000.000000  200000.000000  200000.000000
mean       10.867450       3.547040      78.712437    8580.651700
std         1.506971       0.607708       5.902027     638.671535
min         2.000000       1.000000       6.033333     803.333333
25%        10.000000       3.000000      77.516667    8540.000000
50%        12.000000       4.000000      80.866667    8721.666667
75%        12.000000       4.000000      80.900000    8890.000000
max        12.000000       4.000000      83.566667    9156.666667


,flights_idx,start_base,first_dep,last_arr,total_hours,duties,n_legs,credit,cost
0,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 63, 69]",BASE1,2000-01-01T12:00:00,2000-01-03T14:30:00,50.5,3,12,50.5,5650.0
1,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 63, 105]",BASE1,2000-01-01T12:00:00,2000-01-04T14:30:00,74.5,3,12,74.5,8050.0
2,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 83, 88]",BASE1,2000-01-01T12:00:00,2000-01-03T20:54:00,56.9,3,12,56.9,6290.0
3,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 83, 124]",BASE1,2000-01-01T12:00:00,2000-01-04T20:54:00,80.9,4,12,80.9,8890.0
4,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 87, 124]",BASE1,2000-01-01T12:00:00,2000-01-04T20:54:00,80.9,4,12,80.9,8890.0
5,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 99, 105]",BASE1,2000-01-01T12:00:00,2000-01-04T14:30:00,74.5,4,12,74.5,8250.0
6,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 45, 119, 124]",BASE1,2000-01-01T12:00:00,2000-01-04T20:54:00,80.9,3,12,80.9,8690.0
7,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 73, 83, 88]",BASE1,2000-01-01T12:00:00,2000-01-03T20:54:00,56.9,3,12,56.9,6290.0
8,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 73, 83, 124]",BASE1,2000-01-01T12:00:00,2000-01-04T20:54:00,80.9,4,12,80.9,8890.0
9,"[0, 2, 6, 10, 15, 18, 26, 32, 40, 73, 87, 124]",BASE1,2000-01-01T12:00:00,2000-01-04T20:54:00,80.9,4,12,80.9,8890.0


Saved pickles: /content/pairings_instance1.pkl /content/flights_instance1.pkl
Saved A_index.json and B_index.json (flight->pairings and base->pairings)


In [ ]:
from google.colab import files
uploaded = files.upload()   # Choose your gurobi.lic file

# Move it to where Gurobi expects it
!mkdir -p /root/gurobi
!mv gurobi.lic /root/gurobi/
!export GRB_LICENSE_FILE=/root/gurobi/gurobi.lic

# Verify license
!pip install -q gurobipy
from gurobipy import Model, GRB
m = Model("license_test")
x = m.addVar(name="x")
m.setObjective(x, GRB.MAXIMIZE)
m.optimize()
print("✅ Gurobi license file loaded successfully.")


In [ ]:
# === Prune pairings per flight and run solve_cpp in a loop (auto-shrink K if Gurobi license too small) ===
import os, json, math, shutil, sys, time
import pandas as pd

INSTANCE_DIR = "/content/instance1"   # adjust if needed
PAIRINGS_PKL = "/content/pairings_instance1.pkl"
FLIGHTS_PKL  = "/content/flights_instance1.pkl"
A_INDEX_JSON = "/content/A_index.json"
LIST_OF_BASES = os.path.join(INSTANCE_DIR, "listOfBases.csv")

# sanity checks
for p in [PAIRINGS_PKL, FLIGHTS_PKL, A_INDEX_JSON, LIST_OF_BASES]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Required file not found: {p}")

pairings_df = pd.read_pickle(PAIRINGS_PKL).reset_index(drop=True)
flights_df  = pd.read_pickle(FLIGHTS_PKL).reset_index(drop=True)
with open(A_INDEX_JSON, "r") as fh:
    A_index = json.load(fh)

# make sure cost exists
if 'cost' not in pairings_df.columns:
    PAY_PER_HOUR = 100.0
    OVERNIGHT_COST = 200.0
    pairings_df['cost'] = pairings_df.get('total_hours', pairings_df.get('duration_hr', 0.0)) * PAY_PER_HOUR + pairings_df.get('duties', 0) * OVERNIGHT_COST

# Ensure pairing_id exists (original index-based)
pairings_df['orig_index'] = pairings_df.index.astype(int)
pairings_df['pairing_id_orig'] = pairings_df['orig_index'].apply(lambda x: f"P{x:06d}")

# Create a mapping flight_idx -> list of (pairing_orig_index, cost)
flight_to_pairs = {}
for f_str, plist in A_index.items():
    f = int(f_str)
    lst = []
    for p in plist:
        p_int = int(p)
        if p_int < len(pairings_df):
            lst.append((p_int, float(pairings_df.loc[p_int, 'cost'])))
    # sort by cost
    lst.sort(key=lambda x: x[1])
    flight_to_pairs[f] = lst

# pruning + solver loop
from importlib import import_module
# try to import your solver module (allow either 1662 or solver_1662)
solver_module = None
for modname in ("solver_1662", "1662", "solver_1662_py", "solver1662"):
    try:
        solver_module = import_module(modname)
        break
    except Exception:
        pass
if solver_module is None:
    # fallback: try to import by filename
    solver_path = os.path.join("/", "content", "1662.py")
    if os.path.exists(solver_path):
        import importlib.util
        spec = importlib.util.spec_from_file_location("solver_module_custom", solver_path)
        solver_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(solver_module)
    else:
        raise ImportError("Could not import your solver wrapper (1662.py). Make sure it's in /content or instance folder.")

solve_cpp = getattr(solver_module, "solve_cpp", None)
if solve_cpp is None:
    raise AttributeError("solve_cpp function not found in the solver module. Check 1662.py or solver_1662.py")

# function to write pruned CSVs for a given K
def write_pruned_files(K):
    # pick keep set: union over top-K per flight
    keep_set = set()
    for f, lst in flight_to_pairs.items():
        topk = [p for p,_ in lst[:K]]
        keep_set.update(topk)
    keep_list = sorted(list(keep_set))
    pruned_df = pairings_df.loc[keep_list].reset_index(drop=True).copy()
    # reassign new pairing ids (0..)
    # NOTE: use list comprehension to avoid Index.apply issue
    pruned_df = pruned_df.reset_index(drop=True)
    pruned_df['pairing_id'] = [f"P{idx:06d}" for idx in range(len(pruned_df))]

    # write pairings.csv (fields solver expects: pairing_id, base, duration_hr, cost)
    pairings_csv = os.path.join(INSTANCE_DIR, "pairings.csv")
    if 'start_base' in pruned_df.columns:
        base_col = 'start_base'
    elif 'base' in pruned_df.columns:
        base_col = 'base'
    else:
        # try to infer from first flight in flights_idx
        def infer_base(row):
            if isinstance(row.get('flights_idx'), (list,tuple)):
                first_idx = int(row['flights_idx'][0])
                return flights_df.loc[first_idx, 'dep_airport']
            return ''
        pruned_df['start_base'] = pruned_df.apply(infer_base, axis=1)
        base_col = 'start_base'

    pruned_df['duration_hr'] = pruned_df.get('total_hours', pruned_df.get('duration_hr', 0.0))
    pruned_df[['pairing_id', base_col, 'duration_hr', 'cost']].rename(columns={base_col:'base'}).to_csv(pairings_csv, index=False)

    # build a_ip mapping using new pairing ids
    orig_to_new = {orig: new for new, orig in enumerate(keep_list)}
    rows = []
    for f_idx, lst in flight_to_pairs.items():
        for orig_p, _ in lst:
            if orig_p in orig_to_new:
                new_idx = orig_to_new[orig_p]
                pid = f"P{new_idx:06d}"
                rows.append({"flight_idx": int(f_idx), "pairing_id": pid})
    a_ip_df = pd.DataFrame(rows)
    a_ip_df.to_csv(os.path.join(INSTANCE_DIR, "a_ip.csv"), index=False)

    # write flights_clean.csv and credit_clean.csv if not present (keep existing ones if present)
    flights_clean_csv = os.path.join(INSTANCE_DIR, "flights_clean.csv")
    if not os.path.exists(flights_clean_csv):
        fd = flights_df.copy()
        fd['flight_idx'] = fd.index.astype(int)
        fd['dep_dt'] = fd['dep_dt'].astype(str)
        fd['arr_dt'] = fd['arr_dt'].astype(str)
        fd[['flight_idx','flight_id','dep_airport','arr_airport','dep_dt','arr_dt','duration_hr']].to_csv(flights_clean_csv, index=False)

    # credit file: try to reuse credit_clean.csv if exists, else create a safe non-binding file
    credit_clean = os.path.join(INSTANCE_DIR, "credit_clean.csv")
    if not os.path.exists(credit_clean):
        # give large credits (no binding) as fallback
        bases_df = pd.read_csv(LIST_OF_BASES)
        bases_df.columns = bases_df.columns.str.strip().str.lower()
        base_names = bases_df['airport'].astype(str).str.strip().tolist()
        credit_map = [{"base": b, "allowed_credit": 1e9} for b in base_names]
        pd.DataFrame(credit_map).to_csv(credit_clean, index=False)

    return pruned_df.shape[0], os.path.join(INSTANCE_DIR, "pairings.csv"), os.path.join(INSTANCE_DIR, "a_ip.csv")

# Try K loop
Ks = [25, 15, 10, 5]
success = False
last_exception = None

for K in Ks:
    print(f"\nTrying K = {K} per-flight pruning ...")
    n_pairs, pairings_csv, a_ip_csv = write_pruned_files(K)
    print("Pruned pairings written:", pairings_csv, "count:", n_pairs)
    # call solver
    try:
        start = time.time()
        result = solve_cpp(INSTANCE_DIR,
                           pairings_file="pairings.csv",
                           a_ip_file="a_ip.csv",
                           flights_file="flights_clean.csv",
                           credit_file="credit_clean.csv",
                           time_limit=600,
                           mip_gap=1e-4)
        # solve_cpp may return different signatures; handle either (selected_df, obj) or other
        if isinstance(result, tuple) and len(result) >= 2:
            selected_df, obj = result[0], result[1]
        else:
            # some wrappers return only selected_df
            selected_df = result
            obj = None
        elapsed = time.time() - start
        print(f"Solve succeeded with K={K} in {elapsed:.1f}s; selected rows = {len(selected_df)}; obj = {obj}")
        # save selected file (if dataframe)
        try:
            sel_out = os.path.join(INSTANCE_DIR, "selected_pairings.csv")
            if hasattr(selected_df, "to_csv"):
                selected_df.to_csv(sel_out, index=False)
                print("Saved selected_pairings.csv to", sel_out)
        except Exception:
            pass
        success = True
        break
    except Exception as e:
        print("Solver failed for K =", K, "with exception:", type(e), str(e))
        last_exception = e
        # continue with smaller K

if not success:
    print("\nAll pruning attempts failed. Last exception:")
    import traceback
    traceback.print_exception(type(last_exception), last_exception, last_exception.__traceback__)
    print("\nYou can either upload a full Gurobi license (gurobi.lic) into Colab or try the CBC fallback. "
          "Tell me if you want me to auto-run the CBC fallback now.")
else:
    print("\nDone. If solution found, check /content/instance1/selected_pairings.csv and solver logs above.")


In [ ]:
# Cell 2: make sure solver_1662.py is in python path (it was uploaded). If not, copy it to instance dir.
import shutil, os
src = "/content/1662.py"
dst_dir = INSTANCE_DIR # Use the already defined INSTANCE_DIR
dst_file = os.path.join(dst_dir, "solver_1662.py") # Rename the file
shutil.copyfile(src, dst_file)
print("Copied 1662.py to", dst_file)

In [ ]:
# Cell 3: import the wrapper and run solve_cpp
import sys
sys.path.append(INSTANCE_DIR)
from solver_1662 import solve_cpp, solve_crp   # the file you uploaded; uses gurobipy

# run CPP (adjust time_limit / files if needed)
selected_df, obj = solve_cpp(INSTANCE_DIR,
                             pairings_file="pairings.csv",
                             a_ip_file="a_ip.csv",
                             flights_file="flights_clean.csv",
                             credit_file="credit_clean.csv",
                             time_limit=600,   # seconds
                             mip_gap=1e-4)

print("CPP returned objective:", obj)
print("Selected pairings count:", len(selected_df))
selected_out = os.path.join(INSTANCE_DIR, "selected_pairings.csv")
selected_df.to_csv(selected_out, index=False)
print("Saved selected pairings to:", selected_out)

In [ ]:
# Cell A: Validate CPP solution
import pandas as pd, os, json
INSTANCE_DIR = "/content/instance1"

# load relevant files
pair_sel = pd.read_csv(os.path.join(INSTANCE_DIR, "selected_pairings.csv"))
pairings = pd.read_csv(os.path.join(INSTANCE_DIR, "pairings.csv"))
flights = pd.read_csv(os.path.join(INSTANCE_DIR, "flights_clean.csv"))
a_ip = pd.read_csv(os.path.join(INSTANCE_DIR, "a_ip.csv"))
credit = pd.read_csv(os.path.join(INSTANCE_DIR, "credit_clean.csv"))
bases = pd.read_csv(os.path.join(INSTANCE_DIR, "listOfBases.csv"), header=0)
bases.columns = bases.columns.str.strip().str.lower()

# Map selected pairings (pairing_id matches pairings.csv)
selected_ids = set(pair_sel['pairing_id'].astype(str).tolist())
print("Selected pairing ids sample:", list(selected_ids)[:8])

# 1) Coverage: for each flight, count selected columns covering it
cov = a_ip[a_ip['pairing_id'].isin(selected_ids)].groupby('flight_idx').size().reset_index(name='cover_count')
# join to flights to find uncovered flights
flights['flight_idx'] = flights['flight_idx'].astype(int)
cov_full = flights.merge(cov, how='left', left_on='flight_idx', right_on='flight_idx').fillna(0)
uncovered = cov_full[cov_full['cover_count'] < 1]
print("Total flights:", len(flights), "Covered flights:", len(flights)-len(uncovered), "Uncovered flights:", len(uncovered))
if len(uncovered) > 0:
    display(uncovered.head(10))

# 2) Base credit usage: sum credit/duration of selected pairings per base
# pairings.csv has columns pairing_id, base, duration_hr (or duration_hr field name)
pairings_columns = pairings.columns.tolist()
dur_col = 'duration_hr' if 'duration_hr' in pairings_columns else ('total_hours' if 'total_hours' in pairings_columns else None)
if dur_col is None:
    # try reading the original pickle
    try:
        import pickle
        pr = pd.read_pickle("/content/pairings_instance1.pkl")
        pr['pairing_id'] = pr.index.astype(str).apply(lambda x:f"P{int(x):06d}")
        pr_sel = pr[pr['pairing_id'].isin(selected_ids)]
        base_usage = pr_sel.groupby('start_base')['total_hours'].sum().reset_index(name='used_credit')
    except Exception as e:
        print("Couldn't compute credit usage automatically; please ensure duration column exists.")
        base_usage = pd.DataFrame()
else:
    pr_sel = pairings[pairings['pairing_id'].isin(selected_ids)]
    base_usage = pr_sel.groupby('base')[dur_col].sum().reset_index(name='used_credit')

# Merge with allowed credit
credit.columns = credit.columns.str.strip().str.lower()
if 'base' in credit.columns and ('allowed_credit' in credit.columns or 'credit' in credit.columns):
    allowed_col = 'allowed_credit' if 'allowed_credit' in credit.columns else 'credit'
    credit_map = credit.set_index('base')[allowed_col].to_dict()
    base_usage['allowed_credit'] = base_usage['base'].map(lambda b: float(credit_map.get(b, 1e12)))
    base_usage['pct_used'] = base_usage['used_credit'] / base_usage['allowed_credit'] * 100.0
else:
    base_usage['allowed_credit'] = float(1e12)
    base_usage['pct_used'] = 0.0

print("\nBase credit usage (sample):")
display(base_usage.sort_values('pct_used', ascending=False).head(10))

# Save diagnostic CSVs
base_usage.to_csv(os.path.join(INSTANCE_DIR, "diagnostic_base_usage.csv"), index=False)
cov_full.to_csv(os.path.join(INSTANCE_DIR, "diagnostic_flight_coverage.csv"), index=False)
print("Diagnostics saved to instance folder: diagnostic_base_usage.csv, diagnostic_flight_coverage.csv")


In [ ]:
# Cell A: compute IIS for the CRP model and print problem constraints in IIS
import importlib.util, os, time, pandas as pd, traceback

INSTANCE_DIR = "/content/instance1"
solver_path = "/content/1662.py"

# load the solver module so we can access solve_crp internals if needed
spec = importlib.util.spec_from_file_location("solver_mod", solver_path)
solver_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver_mod)
solve_crp = getattr(solver_mod, "solve_crp", None)

# We need the actual Gurobi model object to compute IIS.
# If solve_crp constructs it internally, we will re-run a modified function that returns the model for diagnostics.
# To avoid editing your file permanently, we will create a small wrapper that calls your function but intercepts the model
# If your solve_crp does not expose the model, we will re-implement the key part: build the same CRP model from CSVs and compute IIS.

# === Build the CRP model here for diagnostics (re-creating what solve_crp does) ===
import gurobipy as gp
from gurobipy import GRB

# load required CSVs
pairings_file = os.path.join(INSTANCE_DIR, "selected_pairings.csv")  # pairings chosen by CPP
crew_file     = os.path.join(INSTANCE_DIR, "crew.csv")
req_crew = 1  # same as used previously

pairings = pd.read_csv(pairings_file)
crew = pd.read_csv(crew_file)
# ensure columns and types
pairings['pairing_id'] = pairings['pairing_id'].astype(str)
crew['crew_id'] = crew['crew_id'].astype(str)

# we need durations per pairing (column names may vary)
if 'duration_hr' in pairings.columns:
    dur_col = 'duration_hr'
elif 'total_hours' in pairings.columns:
    dur_col = 'total_hours'
else:
    # try to compute from first_dep/last_arr if available
    if 'first_dep' in pairings.columns and 'last_arr' in pairings.columns:
        pairings['duration_hr'] = (pd.to_datetime(pairings['last_arr']) - pd.to_datetime(pairings['first_dep'])).dt.total_seconds()/3600.0
        dur_col = 'duration_hr'
    else:
        raise RuntimeError("Cannot find pairing duration column in selected_pairings.csv")

# Build a simple CRP model consistent with your wrapper:
# Variables y[(c,p)] binary if crew c assigned to pairing p
model = gp.Model("CRP_diag")
model.setParam('OutputFlag', 0)
y = {}
for c in crew['crew_id']:
    for p in pairings['pairing_id']:
        y[(c,p)] = model.addVar(vtype=GRB.BINARY, name=f"y_{c}_{p}")

# Each pairing p must have req_crew assigned:
for p in pairings['pairing_id']:
    model.addConstr(gp.quicksum(y[(c,p)] for c in crew['crew_id']) >= req_crew, name=f"cover_{p}")

# Crew hours constraint: sum(duration_p * y_cp) <= max_hours_c
# ensure crew has max_hours column
if 'max_hours' not in crew.columns:
    crew['max_hours'] = 40.0
for _, row in crew.iterrows():
    c = row['crew_id']
    Hc = float(row.get('max_hours', 40.0))
    model.addConstr(gp.quicksum(pairings.loc[pairings['pairing_id']==p, dur_col].values[0] * y[(c,p)] for p in pairings['pairing_id']) <= Hc, name=f"hours_{c}")

# You may also have base/qualification constraints in your wrapper; include basic base matching:
if 'base' in pairings.columns:
    # crew should only be able to take pairings starting from their base (if wrapper enforces this)
    if 'base' in crew.columns:
        for _, r in crew.iterrows():
            c = r['crew_id']; cbase = str(r['base']).strip()
            for p, prow in pairings.iterrows():
                pbase = str(prow['base']).strip()
                if pbase != cbase:
                    # forbid assignment (add constraint y_cp == 0)
                    model.addConstr(y[(c, prow['pairing_id'])] == 0, name=f"base_forbid_{c}_{prow['pairing_id']}")

# Now compute IIS
model.update()
print("Computing IIS (this may take a few seconds)...")
model.computeIIS()
iiss = []
# model.write will create a .ilp with IIS marked; but we can inspect constraints/vars with IIS flags
for constr in model.getConstrs():
    if constr.IISConstr:
        iiss.append(("constr", constr.ConstrName))
for var in model.getVars():
    if var.IISLB or var.IISUB:
        iiss.append(("var", var.VarName))
print("IIS elements found (constraints/vars):", len(iiss))
# print a readable sample of IIS items
for kind, name in iiss[:200]:
    print(kind, ":", name)
# write IIS to file for inspection
model.write(os.path.join(INSTANCE_DIR, "crp_iis.ilp"))
print("IIS written to:", os.path.join(INSTANCE_DIR, "crp_iis.ilp"))


In [ ]:
# Cell B: temporarily relax max_hours and re-run solve_crp to check feasibility root cause
import pandas as pd, os, importlib.util, time

INSTANCE_DIR = "/content/instance1"
crew_csv = os.path.join(INSTANCE_DIR, "crew.csv")
crew_df = pd.read_csv(crew_csv, dtype=str)
# ensure columns
if 'crew_id' not in crew_df.columns:
    raise RuntimeError("crew.csv missing crew_id")
if 'base' not in crew_df.columns:
    crew_df['base'] = crew_df['crew_id'].apply(lambda x: str(x).split('_')[0] if '_' in x else '')

crew_df['max_hours'] = 1e6  # relax hours massively
crew_df.to_csv(crew_csv, index=False)
print("Temporarily set max_hours=1e6 for all crew and wrote crew.csv")

# reload solver and call solve_crp()
spec = importlib.util.spec_from_file_location("solver_mod", "/content/1662.py")
solver_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver_mod)
solve_crp = getattr(solver_mod, "solve_crp", None)
start = time.time()
try:
    res = solve_crp(INSTANCE_DIR,
                    selected_pairings_file="selected_pairings.csv",
                    crew_file="crew.csv",
                    req_crew=1,
                    time_limit=300)
    elapsed = time.time() - start
    print("solve_crp returned after", f"{elapsed:.1f}s")
    if isinstance(res, tuple):
        assign_df = res[0]
    else:
        assign_df = res
    print("Assignments rows:", len(assign_df))
    assign_df.to_csv(os.path.join(INSTANCE_DIR, "crew_assignments_relaxed_hours.csv"), index=False)
    print("Saved crew_assignments_relaxed_hours.csv")
except Exception as e:
    print("solve_crp still failed after relaxing max_hours. Exception:")
    import traceback
    traceback.print_exception(type(e), e, e.__traceback__)


In [ ]:
# Cell C: automatically add reserve crews at busiest base(s) and re-run with normal max_hours
import pandas as pd, os, time, importlib.util
INSTANCE_DIR = "/content/instance1"
pairings = pd.read_csv(os.path.join(INSTANCE_DIR, "selected_pairings.csv"))
bases_usage = pairings.groupby('base').size().reset_index(name='n_p')
bases_usage = bases_usage.sort_values('n_p', ascending=False)
top_base = bases_usage.iloc[0]['base']
print("Top base (will add reserves):", top_base)

crew_csv = os.path.join(INSTANCE_DIR, "crew.csv")
crew_df = pd.read_csv(crew_csv, dtype=str)
max_existing = pd.to_numeric(crew_df['max_hours'], errors='coerce').fillna(40.0).max()
# add R reserve crews
R = 3
new_rows = []
start_id = len(crew_df)
for i in range(R):
    new_rows.append({"crew_id": f"RES_{i+1:02d}", "base": top_base, "max_hours": 40.0})
crew_df = pd.concat([crew_df, pd.DataFrame(new_rows)], ignore_index=True)
crew_df.to_csv(crew_csv, index=False)
print("Added", R, "reserve crews at base", top_base, "and wrote crew.csv")

# re-run solver
spec = importlib.util.spec_from_file_location("solver_mod", "/content/1662.py")
solver_mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver_mod)
solve_crp = getattr(solver_mod, "solve_crp", None)

start = time.time()
res = solve_crp(INSTANCE_DIR,
                selected_pairings_file="selected_pairings.csv",
                crew_file="crew.csv",
                req_crew=1,
                time_limit=300)
elapsed = time.time() - start
if isinstance(res, tuple):
    assign_df = res[0]
else:
    assign_df = res
assign_df.to_csv(os.path.join(INSTANCE_DIR, "crew_assignments_added_reserve.csv"), index=False)
print("Re-run finished in", f"{elapsed:.1f}s", "Assignments saved to crew_assignments_added_reserve.csv")


In [ ]:
# Validation cell
import pandas as pd, os, numpy as np

INSTANCE_DIR = "/content/instance1"
assign_csv = os.path.join(INSTANCE_DIR, "crew_assignments_added_reserve.csv")
pairings_csv = os.path.join(INSTANCE_DIR, "selected_pairings.csv")
crew_csv = os.path.join(INSTANCE_DIR, "crew.csv")
a_ip_csv = os.path.join(INSTANCE_DIR, "a_ip.csv")

assign_df = pd.read_csv(assign_csv)
pairings = pd.read_csv(pairings_csv)
crew = pd.read_csv(crew_csv)
a_ip = pd.read_csv(a_ip_csv)

# normalize names/types
assign_df['crew'] = assign_df['crew'].astype(str)
assign_df['pairing'] = assign_df['pairing'].astype(str)
pairings['pairing_id'] = pairings['pairing_id'].astype(str)

# 1) Coverage: count how many crews were assigned to each pairing
cov_count = assign_df.groupby('pairing').size().reset_index(name='assigned_crews')
# join required crew (we used req_crew=1 earlier)
cov_count['required_crews'] = 1
# find under-covered pairings
under = cov_count[cov_count['assigned_crews'] < cov_count['required_crews']]
print("Pairings with insufficient crew (should be 0):", len(under))
if len(under) > 0:
    display(under.head(20))

# 2) Per-crew hours: sum durations per assigned pairing (pairings must have duration_hr or total_hours)
dur_col = 'duration_hr' if 'duration_hr' in pairings.columns else ('total_hours' if 'total_hours' in pairings.columns else None)
if dur_col is None:
    # attempt to compute
    if 'first_dep' in pairings.columns and 'last_arr' in pairings.columns:
        pairings[dur_col] = (pd.to_datetime(pairings['last_arr']) - pd.to_datetime(pairings['first_dep'])).dt.total_seconds()/3600.0
    else:
        raise RuntimeError("Cannot find pairing duration column in selected_pairings.csv")

# merge durations into assignments
assign_with_dur = assign_df.merge(pairings[['pairing_id', dur_col, 'base']], left_on='pairing', right_on='pairing_id', how='left')
crew_hours = assign_with_dur.groupby('crew')[dur_col].sum().reset_index(name='assigned_hours')

# merge max_hours from crew.csv if present
if 'max_hours' in crew.columns:
    crew_map = crew.set_index('crew_id')['max_hours'].to_dict()
    crew_hours['max_hours'] = crew_hours['crew'].map(lambda x: float(crew_map.get(x, 40.0)))
else:
    crew_hours['max_hours'] = 40.0

crew_hours['pct_used'] = crew_hours['assigned_hours'] / crew_hours['max_hours'] * 100.0
print("\nPer-crew hours summary (top 10 busiest):")
display(crew_hours.sort_values('assigned_hours', ascending=False).head(10))

# 3) Per-base assignment counts
assign_with_base = assign_with_dur.groupby('base').size().reset_index(name='assignments_count')
print("\nAssignments per base:")
display(assign_with_base.sort_values('assignments_count', ascending=False))

# 4) Save diagnostics
crew_hours.to_csv(os.path.join(INSTANCE_DIR, "diag_crew_hours.csv"), index=False)
assign_with_base.to_csv(os.path.join(INSTANCE_DIR, "diag_assign_per_base.csv"), index=False)
print("\nDiagnostics saved: diag_crew_hours.csv, diag_assign_per_base.csv")


In [ ]:
# Robust roster builder: handles missing first_dep/last_arr by reconstructing from pairings pickle + flights
import os, pandas as pd, ast, json
INSTANCE_DIR = "/content/instance1"

# load files
assign_path = os.path.join(INSTANCE_DIR, "crew_assignments_added_reserve.csv")
pairings_csv = os.path.join(INSTANCE_DIR, "selected_pairings.csv")
pairings_pkl = "/content/pairings_instance1.pkl"
flights_pkl = "/content/flights_instance1.pkl"

assign_df = pd.read_csv(assign_path)
# try to load pairings CSV first
if os.path.exists(pairings_csv):
    pairings = pd.read_csv(pairings_csv)
else:
    pairings = None

# load original pairing pickle if available (more detailed)
orig_pairings = None
if os.path.exists(pairings_pkl):
    try:
        orig_pairings = pd.read_pickle(pairings_pkl)
    except Exception:
        orig_pairings = None

# load flights for timestamp reconstruction
if os.path.exists(flights_pkl):
    flights_df = pd.read_pickle(flights_pkl)
else:
    # try flights_clean.csv fallback
    flights_clean_csv = os.path.join(INSTANCE_DIR, "flights_clean.csv")
    flights_df = pd.read_csv(flights_clean_csv)
    flights_df['dep_dt'] = pd.to_datetime(flights_df['dep_dt'])
    flights_df['arr_dt'] = pd.to_datetime(flights_df['arr_dt'])

# If the pairings CSV already contains first_dep/last_arr/duration_hr, use it; otherwise reconstruct
need_recompute = True
if pairings is not None:
    cols = [c.lower() for c in pairings.columns]
    if 'first_dep' in cols or 'first_dep' in pairings.columns:
        need_recompute = False
    if 'last_arr' in cols or 'last_arr' in pairings.columns:
        need_recompute = need_recompute and False
    # but ensure duration exists
    if 'duration_hr' in cols or 'total_hours' in cols:
        pass

# Build a canonical pairings_df that contains pairing_id, flights_idx (list[int]), first_dep, last_arr, duration_hr, base
def extract_pairings_dataframe():
    # prefer original pickle for flights_idx field (more reliable)
    if orig_pairings is not None:
        p = orig_pairings.reset_index(drop=True).copy()
        # ensure pairing_id aligns with selected_pairings ids: Selected used P000000... mapping earlier
        # Create pairing_id format consistent with pruned selected_pairings (if matching by index)
        p['pairing_id'] = p.index.map(lambda x: f"P{int(x):06d}")
        # flights_idx may be list already; if string, parse
        if 'flights_idx' in p.columns:
            def parse_list(x):
                if isinstance(x, (list,tuple)):
                    return [int(i) for i in x]
                s = str(x)
                # try JSON
                try:
                    v = json.loads(s)
                    return [int(i) for i in v]
                except:
                    pass
                # try ast
                try:
                    v = ast.literal_eval(s)
                    return [int(i) for i in v]
                except:
                    return []
            p['flights_idx'] = p['flights_idx'].apply(parse_list)
        else:
            p['flights_idx'] = [[] for _ in range(len(p))]
        # compute first_dep/last_arr/duration
        def compute_times(flist):
            if not flist:
                return (pd.NaT, pd.NaT, 0.0)
            # map flight idx to datetimes in flights_df
            deps = []
            arrs = []
            for fi in flist:
                # some flights_df index may not match numeric id; try locate by 'flight_idx' or by position
                if 'flight_idx' in flights_df.columns:
                    row = flights_df[flights_df['flight_idx']==int(fi)]
                    if row.shape[0] == 0:
                        # try positional
                        try:
                            row = flights_df.iloc[int(fi)]
                            deps.append(pd.to_datetime(row['dep_dt']))
                            arrs.append(pd.to_datetime(row['arr_dt']))
                            continue
                        except Exception:
                            continue
                    deps.append(pd.to_datetime(row.iloc[0]['dep_dt']))
                    arrs.append(pd.to_datetime(row.iloc[0]['arr_dt']))
                else:
                    try:
                        row = flights_df.iloc[int(fi)]
                        deps.append(pd.to_datetime(row['dep_dt']))
                        arrs.append(pd.to_datetime(row['arr_dt']))
                    except Exception:
                        continue
            if not deps:
                return (pd.NaT, pd.NaT, 0.0)
            first_dep = min(deps)
            last_arr = max(arrs)
            duration_hr = (last_arr - first_dep).total_seconds() / 3600.0
            return (first_dep, last_arr, duration_hr)
        computed = p['flights_idx'].apply(compute_times)
        p['first_dep'] = computed.apply(lambda t: t[0])
        p['last_arr'] = computed.apply(lambda t: t[1])
        p['duration_hr'] = computed.apply(lambda t: t[2])
        # ensure base column exists
        if 'start_base' in p.columns:
            p['base'] = p['start_base'].astype(str)
        elif 'base' in p.columns:
            p['base'] = p['base'].astype(str)
        else:
            p['base'] = p['flights_idx'].apply(lambda lst: flights_df.loc[flights_df['flight_idx']==int(lst[0]), 'dep_airport'].values[0] if lst else '')
        # stringify first_dep/last_arr
        p['first_dep'] = pd.to_datetime(p['first_dep'])
        p['last_arr'] = pd.to_datetime(p['last_arr'])
        return p[['pairing_id','flights_idx','base','first_dep','last_arr','duration_hr']].copy()
    else:
        # fallback: read pairings CSV and try to reconstruct using mapping in a_ip.json
        if pairings is None:
            raise RuntimeError("No pairing data available (neither pickle nor CSV). Cannot reconstruct roster.")
        p = pairings.copy()
        # Ensure pairing_id exists
        if 'pairing_id' not in p.columns:
            p = p.reset_index().rename(columns={'index':'pairing_id'})
            p['pairing_id'] = p['pairing_id'].astype(str).apply(lambda x: f"P{int(x):06d}")
        # try to create flights_idx mapping from a_ip mapping
        a_ip = pd.read_csv(os.path.join(INSTANCE_DIR, "a_ip.csv"))
        # build reverse map pairing_id -> list flight_idx
        map_p_to_f = a_ip.groupby('pairing_id')['flight_idx'].apply(lambda s: [int(x) for x in s.tolist()]).to_dict()
        p['flights_idx'] = p['pairing_id'].map(lambda pid: map_p_to_f.get(pid, []))
        # compute times same as above
        def compute_times2(lst):
            if not lst:
                return (pd.NaT, pd.NaT, 0.0)
            deps, arrs = [], []
            for fi in lst:
                try:
                    row = flights_df[flights_df['flight_idx']==int(fi)]
                    if row.shape[0] > 0:
                        deps.append(pd.to_datetime(row.iloc[0]['dep_dt']))
                        arrs.append(pd.to_datetime(row.iloc[0]['arr_dt']))
                    else:
                        r2 = flights_df.iloc[int(fi)]
                        deps.append(pd.to_datetime(r2['dep_dt']))
                        arrs.append(pd.to_datetime(r2['arr_dt']))
                except Exception:
                    continue
            if not deps:
                return (pd.NaT, pd.NaT, 0.0)
            first_dep = min(deps); last_arr = max(arrs)
            return (first_dep, last_arr, (last_arr-first_dep).total_seconds()/3600.0)
        computed = p['flights_idx'].apply(compute_times2)
        p['first_dep'] = computed.apply(lambda t: t[0])
        p['last_arr'] = computed.apply(lambda t: t[1])
        p['duration_hr'] = computed.apply(lambda t: t[2])
        if 'base' not in p.columns:
            p['base'] = p['flights_idx'].apply(lambda lst: flights_df.loc[flights_df['flight_idx']==int(lst[0]), 'dep_airport'].values[0] if lst else '')
        p['first_dep'] = pd.to_datetime(p['first_dep'])
        p['last_arr'] = pd.to_datetime(p['last_arr'])
        return p[['pairing_id','flights_idx','base','first_dep','last_arr','duration_hr']].copy()

pairings_full = extract_pairings_dataframe()

# Merge assignment with pairing timestamps
assign_df['pairing'] = assign_df['pairing'].astype(str)
merged = assign_df.merge(pairings_full, left_on='pairing', right_on='pairing_id', how='left')
# re-order and select useful columns
roster = merged[['crew','pairing','base','duration_hr','first_dep','last_arr']].copy()
roster = roster.sort_values(['crew','first_dep']).reset_index(drop=True)

# Save roster CSV
out_csv = os.path.join(INSTANCE_DIR, "rosters_per_crew.csv")
roster.to_csv(out_csv, index=False)
print("Wrote roster CSV:", out_csv)
display(roster.head(30))